<a href="https://colab.research.google.com/github/giuliovv/bldc_project_work/blob/master/modello_data_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import base64
import requests

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
def get_values_from_github(url_x, url_y):
  X_read = requests.get("https://raw.githubusercontent.com/giuliovv/bldc_project_work/master/data/" + url_x + ".csv").text
  Y_read = requests.get("https://raw.githubusercontent.com/giuliovv/bldc_project_work/master/data/" + url_y + ".csv").text

  # Last value is empty
  X = X_read.split("\n")[:-1]
  Y = Y_read.split("\n")[:-1]

  array = np.array([X, Y]).astype("float")

  df = pd.DataFrame(array.T, columns=["X", "Y"])
  
  return df

In [ ]:
df = get_values_from_github("input", "output")
df.head()

In [ ]:
df.plot()

Una prima situazione interessante:

In [ ]:
df.iloc[:int(4e3)].plot()

Taglio il primo salto a vuoto:

In [ ]:
def cut_df(df, length):
  return df.iloc[int(length):]

In [ ]:
cut_df(df, 4e3).plot()

Pulizia disturbi:

In [ ]:
def clean_disturbances(df, roll_value):
  return pd.concat([df_cut.X.rolling(roll_value).max(), df_cut.Y.rolling(roll_value).max()], axis=1)

In [ ]:
df_cleaned = clean_disturbances(df, 300)
df_cleaned.plot()

In [ ]:
def pipeline(df):
  cutted = cut_df(df, 160e3)
  cleaned = clean_disturbances(cutted, 300)
  df_nona = cleaned.dropna()
  X_np = df_nona.X.values.reshape(-1, 1)
  Y_np = df_nona.Y.values
  return X_np, Y_np

In [ ]:
x_plt, y_plt = pipeline(df)
plt.plot([x_plt.reshape(-1), y_plt])

## Prova 1: Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
X, y = pipeline(df)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [ ]:
reg = LinearRegression().fit(X_train, y_train)

In [ ]:
reg.score(X_test, y_test)

In [ ]:
reg.coef_

In [ ]:
previsione = reg.predict(X[:200000])

In [ ]:
plt.plot(y[:200000])
plt.plot(previsione)

Test 

In [ ]:
df_test = get_values_from_github("input_test", "output_test")
X, y = pipeline(df_test)

In [ ]:
reg.score(X, y)